In [1]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
from dataset import *
from wrapper import *
from utils import *
from models import *

In [2]:
train_df = pd.read_csv('../data/train.csv', sep='\t')

model_name = 'hfl/chinese-macbert-base'
ner_model_name = 'uer/roberta-base-finetuned-cluener2020-chinese'

train_dataset_config = {
    'model_name':model_name,
    'aux_model_name':ner_model_name,
    'maxlength':64,
    'train_val_split':-1,
    'test':False, 
    'remove_username':False,
    'remove_punctuation':False, 
    'to_simplified':False, 
    'emoji_to_text':False, 
}

train = DatasetWithAuxiliaryEmbeddings(df=train_df, **train_dataset_config)
train.tokenize()
train.construct_dataset()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\holaj\AppData\Local\Temp\jieba.cache
Loading model cost 0.633 seconds.
Prefix dict has been built successfully.
d:\Develop\chinese-grammar-error-detection\notebooks\..\dataset.py:162: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  indexed_value = torch.tensor(value[index]).squeeze()


In [3]:
model = BertWithNER(bert_model=model_name, ner_model=ner_model_name)
model.load_state_dict(torch.load('../ner_run_v2/fold0/checkpoint-4950/pytorch_model.bin'))

Some weights of the model checkpoint at hfl/chinese-macbert-base were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at uer/roberta-base-finetuned-cluener2020-chinese were not used when initializi

<All keys matched successfully>

In [4]:
arguments = AdversarialTrainingArguments(
    output_dir='./eval', 
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=16, 
    num_train_epochs=3,
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",  # save checkpoint at each epoch
    learning_rate=1e-5, 
    load_best_model_at_end=True,
    epsilon=0, 
    alpha=.3, 
    gamma=1, 
)

trainer = AdversarialTrainer(
    model=model, 
    args=arguments, 
    eval_dataset=train.dataset['train'], 
    tokenizer=train.tokenizer, 
    compute_metrics=compute_metrics
)

In [5]:
prediction_output = trainer.predict(train.dataset['train'])

The following columns in the test set don't have a corresponding argument in `BertWithNER.forward` and have been ignored: token_type_ids, labels. If token_type_ids, labels are not expected by `BertWithNER.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 45248
  Batch size = 16


KeyError: 'labels'

In [26]:
from copy import copy

eval_df = copy(train_df)
eval_df['prediction'] = np.argmax(prediction_output.predictions, 1)
eval_df = eval_df[['id', 'label', 'prediction', 'text']]

mislabelled_df = eval_df[eval_df['label'] != eval_df['prediction']]
mislabelled_df.to_csv('mislabelled-mini-with-ner.csv', encoding='utf-8')